In [91]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

#### Data loading

In [ ]:
df = pd.read_csv("main_df_after_merging.csv")
df.shape

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_64915/32431068.py:1: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,145,150,151,156,157,158,159,160,161,162,163,164,165,166,167,174,175,176,181,184,209,210,211,212,213,214,215,238,241,261,262,263,264,265,266,287,289,290,299,300) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/mahdie/Documents/1.PhysioAi/6. Functional test/2.My_script/data/main_df_after_merging.csv")


(37143, 315)

All variables
| variable name  | meaning |
|---|---|
|ws010_ |	first walking speed result|
|ws011_ |	first walking speed time|
|ws012_ |	second walking speed result|
|ws013_ |	second walking speed time|
|cs004_ |	chair rise result|
|cs008_ |	time 5 chair rise|
|gs006_|	maxgrip left one|
|gs007_|	maxgrip left two|
|gs008_|	maxgrip right one|
|gs009_|	maxgrip right two|
|dn002_|    MOB|
|dn003_|    YOB|
|dn042_|    female|
|ph006d8| OA|
|ph010d1| pain joint|
|ph012_ | weight|
|ph013_ | height|
|ph044_ | eyesight_close |
|ph046_ | hearing |

##### Create variable "age"

In [93]:
# Wave 2 is 2007
reference_year = 2007
df['age'] = reference_year - df['YOB']

##### Plausible Height and Weight

Consider weight > 10 and height > 100

In [94]:
def clean_height_weight(df):
    print("Initial dataset size:", len(df))
    
    df = df.loc[df['weight'] > 10.0]
    print("After filtering plausible weight:", len(df))
    

    df = df.loc[df['height'] > 100.0]
    
    df.loc[(df['height'] > 1.0) & (df['height'] < 2.0), 'height'] *= 100
    print("After filtering and correcting height:", len(df))
    
    return df
df_raw = df.copy()
df = clean_height_weight(df)

Initial dataset size: 37143
After filtering plausible weight: 36504
After filtering and correcting height: 15864


In [95]:
excluded_hw = df_raw[~df_raw['mergeid'].isin(df['mergeid'])]
print("Excluded due to implausible/missing weight or height:", excluded_hw.shape)

Excluded due to implausible/missing weight or height: (21279, 316)


##### Create variable "bmi"

In [96]:
df['bmi'] = df['weight'] / ((df['height'] / 100) ** 2)

##### Compute the maximum grip strength across all measurements

In [97]:
df['maxgrip'] = df[['maxgrip_left_one', 'maxgrip_left_two', 'maxgrip_right_one', 'maxgrip_right_two']].max(axis=1)

##### Compute the average walking speed across two attempts

In [98]:
df['walking'] = df[['first_walking_speed_time', 'second_walking_speed_time']].mean(axis=1)

##### Other variables

In [99]:
df['chair'] = df['time_5_chair_rise']
df['OA_conserv'] = np.logical_and(df['OA'], df['pain_joint'].astype(bool)).astype(np.int8)
df['activity'] = 2 * (4 - df['br015_']) + (4 - df['br016_'])
df['female'] = df['female'].astype(int)

In [100]:
df['chair'].sort_values(ascending=False).shape

(15864,)

##### Create df_anno

In [101]:
df_anno = df.drop_duplicates(subset='mergeid', keep='first')
df_anno.set_index('mergeid', inplace=True, drop=True)

df_anno_raw = df_anno.copy()

In [102]:
print("df:", df.shape, "df_anno:", df_anno.shape)

df: (15864, 322) df_anno: (15864, 321)


##### Categorizing & Identifying Control Groups; OA, diabetes, hypertension

In [103]:
df_anno['Hypertension'] = pd.to_numeric(df_anno['ph006d2'])
df_anno['Diabetes'] = pd.to_numeric(df_anno['ph006d5'])

In [104]:
df_anno["control"] = 0
df_anno.loc[(df_anno["OA_conserv"] == 0) & (df_anno["Hypertension"] == 0) & (df_anno["Diabetes"] == 0), "control"] = 1

In [105]:
display(df_anno['Hypertension'].value_counts())
display(df_anno['Diabetes'].value_counts())
display(df_anno['OA_conserv'].value_counts())

Hypertension
0.0    10340
1.0     5496
Name: count, dtype: int64

Diabetes
0.0    14169
1.0     1667
Name: count, dtype: int64

OA_conserv
0    12988
1     2876
Name: count, dtype: int64

In [106]:
display(df_anno[['Hypertension', 'OA_conserv', 'Diabetes']].value_counts(sort=False))

Hypertension  OA_conserv  Diabetes
0.0           0           0.0         8294
                          1.0          535
              1           0.0         1367
                          1.0          144
1.0           0           0.0         3450
                          1.0          702
              1           0.0         1058
                          1.0          286
Name: count, dtype: int64

## data exploration

##### Select features

In [107]:
df_anno.shape

(15864, 324)

In [108]:
df_anno = df_anno.drop_duplicates()
print(df_anno.shape)
_df_anno = df_anno.copy()

(15864, 324)


##### Create df_feature

In [109]:
# features = ['bmi', 'weight', 'age', 'OA_conserv', 'female', 'hearing', 'eyesight_close', 'Hypertension', 'Diabetes']
features = ['age', 'bmi', 'female', 'educational_level', 'OA_conserv', 'Hypertension', 'Diabetes']
df_features = df_anno.loc[:, features].copy()
df_features = df_features.dropna(axis=0)

In [110]:
excluded_features = df_anno[~df_anno.index.isin(df_features.index)]
print("Excluded due to missing features:", excluded_features.shape)


Excluded due to missing features: (43, 324)


In [111]:
df_features.shape

(15821, 7)

In [112]:
df_features.isna().sum()

age                  0
bmi                  0
female               0
educational_level    0
OA_conserv           0
Hypertension         0
Diabetes             0
dtype: int64

In [113]:
excluded_all = df_raw[~df_raw['mergeid'].isin(df_features.index)]
print("Total excluded cases:", excluded_all.shape)

Total excluded cases: (21322, 316)


In [ ]:
excluded_all.to_csv("excluded_cases.csv")
df_features.to_csv("included_cases.csv")

##### Create data with outcomes and features

In [115]:
data = pd.concat([df_features, df_anno.loc[:, ['maxgrip', 'chair', 'walking',]]], axis=1)

In [116]:
data.shape

(15864, 10)

In [117]:
data = data.dropna(subset=features)

In [118]:
data.shape

(15821, 10)

In [119]:
data.isna().sum()

age                      0
bmi                      0
female                   0
educational_level        0
OA_conserv               0
Hypertension             0
Diabetes                 0
maxgrip               1407
chair                 5308
walking              14396
dtype: int64

In [120]:
print("df:", df.shape)
print("df_anno:", df_anno.shape)
print("df_features:", df_features.shape)
print("data:", data.shape)

df: (15864, 322)
df_anno: (15864, 324)
df_features: (15821, 7)
data: (15821, 10)


In [121]:
features = ['age', 'bmi', 'female', 'educational_level', 'OA_conserv', 'Hypertension', 'Diabetes']
outcomes = ['maxgrip', 'chair', 'walking']
print(data.shape)

(15821, 10)


In [122]:
print(data['female'].value_counts())
print("------------------------------------------------------------")
print(data['OA_conserv'].value_counts())
print("------------------------------------------------------------")
print(data['Hypertension'].value_counts())
print("------------------------------------------------------------")
print(data['Diabetes'].value_counts())

female
1.0    8716
0.0    7105
Name: count, dtype: int64
------------------------------------------------------------
OA_conserv
0.0    12970
1.0     2851
Name: count, dtype: int64
------------------------------------------------------------
Hypertension
0.0    10335
1.0     5486
Name: count, dtype: int64
------------------------------------------------------------
Diabetes
0.0    14158
1.0     1663
Name: count, dtype: int64


In [123]:
data.chair.value_counts()

chair
10.000000    272
9.000000     212
8.000000     208
12.000000    207
11.000000    176
            ... 
9.140000       1
16.040001      1
15.320000      1
24.580000      1
2.650000       1
Name: count, Length: 2552, dtype: int64

In [124]:
mean_age_per_chair = data.groupby('chair')['age'].mean()
print(mean_age_per_chair)


chair
0.000000     55.000000
0.050000     46.500000
0.060000     61.666667
0.060000     57.000000
0.070000     61.000000
               ...    
89.000000    56.000000
90.000000    73.000000
91.000000    65.000000
91.150002    72.000000
98.000000    65.000000
Name: age, Length: 2552, dtype: float64


#### Summery table

In [125]:
features = ['age', 'bmi', 'female', 'educational_level', 'OA_conserv', 'Hypertension', 'Diabetes']
outcomes = ['maxgrip', 'chair', 'walking']

# Create separate datasets for each outcome
datasets = {outcome: data[features + [outcome]].dropna() for outcome in outcomes}

for outcome, df_outcome in datasets.items():
    print(f"Dataset for {outcome}: {df_outcome.shape}")


Dataset for maxgrip: (14414, 8)
Dataset for chair: (10513, 8)
Dataset for walking: (1425, 8)


In [126]:
df_maxgrip = datasets['maxgrip']
df_chair = datasets['chair']
df_walking = datasets['walking']

print("Valid samples per dataset:")
print("data Maxgrip:", df_maxgrip.shape)
print("data Chair Stand:", df_chair.shape)
print("data Walking Speed:", df_walking.shape)

Valid samples per dataset:
data Maxgrip: (14414, 8)
data Chair Stand: (10513, 8)
data Walking Speed: (1425, 8)


In [127]:
df_maxgrip.age.describe()

count    14414.000000
mean        62.993270
std         10.068992
min         15.000000
25%         55.000000
50%         61.000000
75%         70.000000
max         99.000000
Name: age, dtype: float64

In [128]:
df_chair.age.describe()

count    10513.000000
mean        59.708266
std          7.457453
min         15.000000
25%         54.000000
50%         59.000000
75%         65.000000
max         96.000000
Name: age, dtype: float64

In [129]:
df_walking.age.describe()

count    1425.000000
mean       80.066667
std         3.985813
min        60.000000
25%        77.000000
50%        79.000000
75%        82.000000
max        98.000000
Name: age, dtype: float64

##### disease_group

- **0**: No disease  
- **1**:  
  - Diabetes only  
  - Hypertension only  
  - OA only  
- **2**:  
  - Diabetes + Hypertension  
  - Diabetes + OA  
  - Hypertension + OA  
- **3**: Diabetes + Hypertension + OA 


In [130]:
# Create a new categorical variable 'disease_group' based on conditions in 'data'
data['disease_group'] = 0  # Default: No disease

# Assign groups based on disease conditions
data.loc[(data['Diabetes'] == 1) & (data['Hypertension'] == 0) & (data['OA_conserv'] == 0), 'disease_group'] = 1
data.loc[(data['Diabetes'] == 0) & (data['Hypertension'] == 1) & (data['OA_conserv'] == 0), 'disease_group'] = 1
data.loc[(data['Diabetes'] == 0) & (data['Hypertension'] == 0) & (data['OA_conserv'] == 1), 'disease_group'] = 1

data.loc[(data['Diabetes'] == 1) & (data['Hypertension'] == 1) & (data['OA_conserv'] == 0), 'disease_group'] = 2
data.loc[(data['Diabetes'] == 1) & (data['Hypertension'] == 0) & (data['OA_conserv'] == 1), 'disease_group'] = 2
data.loc[(data['Diabetes'] == 0) & (data['Hypertension'] == 1) & (data['OA_conserv'] == 1), 'disease_group'] = 2

data.loc[(data['Diabetes'] == 1) & (data['Hypertension'] == 1) & (data['OA_conserv'] == 1), 'disease_group'] = 3

# Convert disease_group to categorical
data['disease_group'] = data['disease_group'].astype(int)

In [131]:
disease_counts = data['disease_group'].value_counts()
print(disease_counts)

disease_group
0    8290
1    5347
2    1899
3     285
Name: count, dtype: int64


In [132]:
valid_samples = {
    "Maxgrip": data.dropna(subset=['maxgrip'])['disease_group'].value_counts(),
    "Chair Stand": data.dropna(subset=['chair'])['disease_group'].value_counts(),
    "Walking Speed": data.dropna(subset=['walking'])['disease_group'].value_counts()
}

valid_samples_df = pd.DataFrame(valid_samples)
valid_samples_df

,Maxgrip,Chair Stand,Walking Speed
disease_group,,,
0,7612,6117,533
1,4896,3314,619
2,1681,991,231
3,225,91,42


##### Summery

In [133]:
print("data: ", data.shape)
print("maxgrip: ", df_maxgrip.shape)
print("chair: ", df_chair.shape)
print("walking: ", df_walking.shape)

data:  (15821, 11)
maxgrip:  (14414, 8)
chair:  (10513, 8)
walking:  (1425, 8)


In [134]:
data

,age,bmi,female,educational_level,OA_conserv,Hypertension,Diabetes,maxgrip,chair,walking,disease_group
mergeid,,,,,,,,,,,
AT-004234-02,53.0,30.717400,1.0,1.0,0.0,0.0,1.0,37.0,6.00,NaN,1
AT-016392-01,61.0,23.588329,1.0,1.0,0.0,0.0,0.0,34.0,13.20,NaN,0
AT-017298-01,64.0,25.381469,0.0,2.0,0.0,1.0,0.0,55.0,9.40,NaN,1
AT-026212-02,59.0,24.337480,0.0,1.0,0.0,0.0,0.0,52.0,22.00,NaN,0
AT-057442-01,66.0,31.221304,1.0,1.0,0.0,1.0,0.0,NaN,10.84,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
SE-994503-02,77.0,24.056935,0.0,2.0,0.0,0.0,0.0,43.0,NaN,2.105,0
SE-996850-01,61.0,26.827421,0.0,1.0,0.0,0.0,0.0,61.0,7.03,NaN,0
SE-996850-02,61.0,23.323418,1.0,1.0,0.0,1.0,0.0,32.0,13.94,NaN,1


In [135]:
data.isna().sum()

age                      0
bmi                      0
female                   0
educational_level        0
OA_conserv               0
Hypertension             0
Diabetes                 0
maxgrip               1407
chair                 5308
walking              14396
disease_group            0
dtype: int64

In [136]:
# value count of ech columns in data
features = ['age', 'bmi', 'female', 'educational_level', 'OA_conserv', 'Hypertension', 'Diabetes']
for i in features: 
    print(data[i].value_counts())
    print("------------------------------------------------------------")


age
51.0     924
52.0     915
57.0     611
59.0     599
61.0     597
        ... 
15.0       1
100.0      1
30.0       1
98.0       1
31.0       1
Name: count, Length: 72, dtype: int64
------------------------------------------------------------
bmi
24.221453    110
25.711662    104
27.681661    101
25.951557     90
27.343750     85
            ... 
28.577961      1
17.715421      1
24.382717      1
27.776912      1
22.591438      1
Name: count, Length: 2821, dtype: int64
------------------------------------------------------------
female
1.0    8716
0.0    7105
Name: count, dtype: int64
------------------------------------------------------------
educational_level
0.0    6569
1.0    5654
2.0    3022
3.0     576
Name: count, dtype: int64
------------------------------------------------------------
OA_conserv
0.0    12970
1.0     2851
Name: count, dtype: int64
------------------------------------------------------------
Hypertension
0.0    10335
1.0     5486
Name: count, dtype: int64
--

In [137]:
# value count of ech columns in data
features = ['age', 'bmi', 'female', 'educational_level', 'OA_conserv', 'Hypertension', 'Diabetes']
for i in features:     
    print(data[i].describe())
    print("------------------------------------------------------------")

count    15821.00000
mean        63.48739
std         10.38545
min         15.00000
25%         55.00000
50%         62.00000
75%         71.00000
max        100.00000
Name: age, dtype: float64
------------------------------------------------------------
count    15821.000000
mean        26.769211
std          4.703205
min         10.816658
25%         23.808690
50%         26.218821
75%         29.097746
max        157.394159
Name: bmi, dtype: float64
------------------------------------------------------------
count    15821.000000
mean         0.550913
std          0.497417
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: female, dtype: float64
------------------------------------------------------------
count    15821.000000
mean         0.848619
std          0.853802
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          3.000000
Name: educational_level, dtype: float64


##### Create maxchair and maxwalk

In [138]:
data.isna().sum()

age                      0
bmi                      0
female                   0
educational_level        0
OA_conserv               0
Hypertension             0
Diabetes                 0
maxgrip               1407
chair                 5308
walking              14396
disease_group            0
dtype: int64

In [139]:
features = ['age', 'bmi', 'female', 'educational_level', 'OA_conserv', 'Hypertension', 'Diabetes']
outcomes = ['maxgrip', 'chair', 'walking']

maxchair = data[features + ['maxgrip', 'chair']].dropna()

maxwalk = data[features + ['maxgrip', 'walking']].dropna()

print("Dataset maxchair (Maxgrip + Chair):", maxchair.shape)
print("Dataset maxwalk (Maxgrip + Walking):", maxwalk.shape)


Dataset maxchair (Maxgrip + Chair): (10359, 9)
Dataset maxwalk (Maxgrip + Walking): (1361, 9)


In [140]:
# Restrict maxchair to age < 75
maxchair = maxchair[maxchair['age'] < 75].copy()

In [141]:
# Restrict maxwalk to age >= 75
maxwalk= maxwalk[maxwalk['age'] >= 75].copy()

##### Remove ages under 50

In [142]:
##### Remove ages under 50
new_maxchair = maxchair[maxchair['age'] >= 50].copy()
new_maxchair.shape

(9753, 9)

In [143]:
maxchair.shape

(10196, 9)

In [144]:
maxchair = new_maxchair

##### Overlap between maxchair and maxwalk:

In [145]:
overlap_cases_max = maxchair.index.intersection(maxwalk.index)
print(f"Number of overlapping cases between maxchair and maxwalk: {len(overlap_cases_max)}")

Number of overlapping cases between maxchair and maxwalk: 0


##### Overlap between Chair and Walk:

In [146]:
overlap_cases = df_chair.index.intersection(df_walking.index)
print(f"Number of overlapping cases: {len(overlap_cases)}")

Number of overlapping cases: 0


##### Describe of Age:

In [147]:
datasets = {
    "Maxgrip": df_maxgrip,
    "Chair": df_chair,
    "Walking": df_walking,
    "Maxchair": maxchair,
    "Maxwalk": maxwalk
}

for name, df in datasets.items():
    print(f"\nAge Statistics for {name}:\n{df.age.describe()}")
    print("-" * 50)



Age Statistics for Maxgrip:
count    14414.000000
mean        62.993270
std         10.068992
min         15.000000
25%         55.000000
50%         61.000000
75%         70.000000
max         99.000000
Name: age, dtype: float64
--------------------------------------------------

Age Statistics for Chair:
count    10513.000000
mean        59.708266
std          7.457453
min         15.000000
25%         54.000000
50%         59.000000
75%         65.000000
max         96.000000
Name: age, dtype: float64
--------------------------------------------------

Age Statistics for Walking:
count    1425.000000
mean       80.066667
std         3.985813
min        60.000000
25%        77.000000
50%        79.000000
75%        82.000000
max        98.000000
Name: age, dtype: float64
--------------------------------------------------

Age Statistics for Maxchair:
count    9753.000000
mean       60.091357
std         6.720434
min        50.000000
25%        54.000000
50%        59.000000
75%     

#### Summery for paper:

##### maxchair:

In [148]:
maxchair.shape

(9753, 9)

In [149]:
maxchair['disease_category'] = 'No disease'
maxchair.loc[(maxchair['OA_conserv'] == 1) & (maxchair['Hypertension'] == 0) & (maxchair['Diabetes'] == 0), 'disease_category'] = 'Only OA'
maxchair.loc[(maxchair['OA_conserv'] == 0) & (maxchair['Hypertension'] == 1) & (maxchair['Diabetes'] == 0), 'disease_category'] = 'Only HT'
maxchair.loc[(maxchair['OA_conserv'] == 0) & (maxchair['Hypertension'] == 0) & (maxchair['Diabetes'] == 1), 'disease_category'] = 'Only Diabetes'
maxchair.loc[(maxchair['OA_conserv'] == 1) & (maxchair['Hypertension'] == 1) & (maxchair['Diabetes'] == 0), 'disease_category'] = 'OA and HT'
maxchair.loc[(maxchair['OA_conserv'] == 1) & (maxchair['Hypertension'] == 0) & (maxchair['Diabetes'] == 1), 'disease_category'] = 'OA and Diab'
maxchair.loc[(maxchair['OA_conserv'] == 0) & (maxchair['Hypertension'] == 1) & (maxchair['Diabetes'] == 1), 'disease_category'] = 'HT and Diab'
maxchair.loc[(maxchair['OA_conserv'] == 1) & (maxchair['Hypertension'] == 1) & (maxchair['Diabetes'] == 1), 'disease_category'] = 'All three diseases'


summary_dict = {
    'Variable': ['Number of cases', 'Age; mean (SD)', 'Age; n (%)', 'Sex; n, (%)', 'BMI; mean (SD)',
                 'Educational Level - Low (%)', 'Educational Level - Medium (%)',
                 'Educational Level - High (%)', 'Educational Level - Other (%)',
                 'Grip strength (kg); mean (SD)', 'Maxgrip; n (%)', 'Missing grip strength (n)',
                 '5 Chairs stands (sec); mean (SD)', 'Chair stand; n (%)', 'Missing 5 Chairs stands (n)',
                 'Total missing values (n)']
}


disease_groups = maxchair['disease_category'].unique()


for group in disease_groups:
    subset = maxchair[maxchair['disease_category'] == group]
    num_cases = len(subset)

    # Age 
    age_mean_sd = f"{subset['age'].mean():.1f} ± {subset['age'].std():.1f}" if num_cases > 0 else "NaN"
    age_n = f"{num_cases - subset['age'].isna().sum()} ({((num_cases - subset['age'].isna().sum()) / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    # Female
    female_n = f"{subset['female'].sum()} ({(subset['female'].sum() / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    # BMI
    bmi_mean_sd = f"{subset['bmi'].mean():.1f} ± {subset['bmi'].std():.1f}" if num_cases > 0 else "NaN"

    # Educational level counts (%)
    def edu_percent(level):
        count = (subset['educational_level'] == level).sum()
        return f"{count} ({(count / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    edu_low = edu_percent(0)
    edu_medium = edu_percent(1)
    edu_high = edu_percent(2)
    edu_other = edu_percent(3)

    # Grip strength
    maxgrip_mean_sd = f"{subset['maxgrip'].mean():.1f} ± {subset['maxgrip'].std():.1f}" if num_cases > 0 else "NaN"
    missing_maxgrip = subset['maxgrip'].isna().sum()
    maxgrip_n = f"{num_cases - missing_maxgrip} ({((num_cases - missing_maxgrip) / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    # Chair stand
    chair_mean_sd = f"{subset['chair'].mean():.1f} ± {subset['chair'].std():.1f}" if num_cases > 0 else "NaN"
    missing_chair = subset['chair'].isna().sum()
    chair_n = f"{num_cases - missing_chair} ({((num_cases - missing_chair) / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    total_missing = missing_maxgrip + missing_chair

    summary_dict[group] = [
        num_cases, age_mean_sd, age_n, female_n, bmi_mean_sd,
        edu_low, edu_medium, edu_high, edu_other,
        maxgrip_mean_sd, maxgrip_n, missing_maxgrip,
        chair_mean_sd, chair_n, missing_chair,
        total_missing
    ]


max_len = max(len(v) for v in summary_dict.values())
for k in summary_dict:
    summary_dict[k] += [""] * (max_len - len(summary_dict[k]))

summary_table_maxchair = pd.DataFrame(summary_dict)
summary_table_maxchair


,Variable,Only Diabetes,No disease,Only HT,HT and Diab,All three diseases,Only OA,OA and Diab,OA and HT
0,Number of cases,289,5617,2129,377,88,707,55,491
1,Age; mean (SD),62.2 ± 6.5,58.9 ± 6.5,61.5 ± 6.7,63.3 ± 6.3,63.5 ± 6.9,60.3 ± 6.6,61.5 ± 6.7,62.9 ± 6.6
2,Age; n (%),289 (100.0%),5617 (100.0%),2129 (100.0%),377 (100.0%),88 (100.0%),707 (100.0%),55 (100.0%),491 (100.0%)
3,"Sex; n, (%)",105.0 (36.3%),2892.0 (51.5%),1083.0 (50.9%),180.0 (47.7%),58.0 (65.9%),470.0 (66.5%),33.0 (60.0%),316.0 (64.4%)
4,BMI; mean (SD),27.9 ± 4.5,25.9 ± 4.0,27.9 ± 5.3,29.8 ± 4.7,32.3 ± 5.1,26.2 ± 4.0,28.9 ± 5.0,28.4 ± 4.5
5,Educational Level - Low (%),129 (44.6%),1898 (33.8%),797 (37.4%),154 (40.8%),41 (46.6%),294 (41.6%),30 (54.5%),226 (46.0%)
6,Educational Level - Medium (%),103 (35.6%),2240 (39.9%),832 (39.1%),149 (39.5%),28 (31.8%),260 (36.8%),14 (25.5%),180 (36.7%)
7,Educational Level - High (%),47 (16.3%),1373 (24.4%),454 (21.3%),64 (17.0%),12 (13.6%),124 (17.5%),10 (18.2%),69 (14.1%)
8,Educational Level - Other (%),10 (3.5%),106 (1.9%),46 (2.2%),10 (2.7%),7 (8.0%),29 (4.1%),1 (1.8%),16 (3.3%)
9,Grip strength (kg); mean (SD),38.0 ± 10.9,38.2 ± 11.5,37.7 ± 11.7,36.5 ± 11.4,31.7 ± 11.5,33.6 ± 11.6,33.2 ± 11.8,32.7 ± 11.2


##### maxwalk

In [150]:
maxwalk.shape

(1331, 9)

In [151]:
only_maxgrip = maxwalk[(maxwalk['maxgrip'].notna()) & (maxwalk['walking'].isna())]
only_walking = maxwalk[(maxwalk['walking'].notna()) & (maxwalk['maxgrip'].isna())]

print(f"Cases with Maxgrip but missing Walking speed: {len(only_maxgrip)}")
print(f"Cases with Walking speed but missing Maxgrip: {len(only_walking)}")


Cases with Maxgrip but missing Walking speed: 0
Cases with Walking speed but missing Maxgrip: 0


In [152]:
maxwalk['disease_category'] = 'No disease'
maxwalk.loc[(maxwalk['OA_conserv'] == 1) & (maxwalk['Hypertension'] == 0) & (maxwalk['Diabetes'] == 0), 'disease_category'] = 'Only OA'
maxwalk.loc[(maxwalk['OA_conserv'] == 0) & (maxwalk['Hypertension'] == 1) & (maxwalk['Diabetes'] == 0), 'disease_category'] = 'Only HT'
maxwalk.loc[(maxwalk['OA_conserv'] == 0) & (maxwalk['Hypertension'] == 0) & (maxwalk['Diabetes'] == 1), 'disease_category'] = 'Only Diabetes'
maxwalk.loc[(maxwalk['OA_conserv'] == 1) & (maxwalk['Hypertension'] == 1) & (maxwalk['Diabetes'] == 0), 'disease_category'] = 'OA and HT'
maxwalk.loc[(maxwalk['OA_conserv'] == 1) & (maxwalk['Hypertension'] == 0) & (maxwalk['Diabetes'] == 1), 'disease_category'] = 'OA and Diab'
maxwalk.loc[(maxwalk['OA_conserv'] == 0) & (maxwalk['Hypertension'] == 1) & (maxwalk['Diabetes'] == 1), 'disease_category'] = 'HT and Diab'
maxwalk.loc[(maxwalk['OA_conserv'] == 1) & (maxwalk['Hypertension'] == 1) & (maxwalk['Diabetes'] == 1), 'disease_category'] = 'All three diseases'


summary_dict = {
    'Variable': ['Number of cases', 'Age; mean (SD)', 'Age; n (%)', 'Sex; n, (%)', 'BMI; mean (SD)',
                 'Educational Level - Low (%)', 'Educational Level - Medium (%)',
                 'Educational Level - High (%)', 'Educational Level - Other (%)',
                 'Grip strength (kg); mean (SD)', 'Maxgrip; n (%)', 'Missing grip strength (n)',
                 'Walking speed (sec); mean (SD)', 'Walking speed; n (%)', 'Missing walking speed (n)',
                 'Total missing values (n)']
}


disease_groups = maxwalk['disease_category'].unique()


for group in disease_groups:
    subset = maxwalk[maxwalk['disease_category'] == group]
    num_cases = len(subset)

    # Age 
    age_mean_sd = f"{subset['age'].mean():.1f} ± {subset['age'].std():.1f}" if num_cases > 0 else "NaN"
    age_n = f"{num_cases - subset['age'].isna().sum()} ({((num_cases - subset['age'].isna().sum()) / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    # Female
    female_n = f"{subset['female'].sum()} ({(subset['female'].sum() / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    # BMI
    bmi_mean_sd = f"{subset['bmi'].mean():.1f} ± {subset['bmi'].std():.1f}" if num_cases > 0 else "NaN"

    # Educational level counts (%)
    def edu_percent(level):
        count = (subset['educational_level'] == level).sum()
        return f"{count} ({(count / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    edu_low = edu_percent(0)
    edu_medium = edu_percent(1)
    edu_high = edu_percent(2)
    edu_other = edu_percent(3)

    # Grip strength
    maxgrip_mean_sd = f"{subset['maxgrip'].mean():.1f} ± {subset['maxgrip'].std():.1f}" if num_cases > 0 else "NaN"
    missing_maxgrip = subset['maxgrip'].isna().sum()
    maxgrip_n = f"{num_cases - missing_maxgrip} ({((num_cases - missing_maxgrip) / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    # Walking speed
    walking_mean_sd = f"{subset['walking'].mean():.1f} ± {subset['walking'].std():.1f}" if num_cases > 0 else "NaN"
    missing_walking = subset['walking'].isna().sum()
    valid_walking = num_cases - missing_walking
    walking_n = f"{valid_walking} ({(valid_walking / num_cases) * 100:.1f}%)" if num_cases > 0 else "NaN"

    total_missing = missing_maxgrip + missing_walking


    summary_dict[group] = [
        num_cases, age_mean_sd, age_n, female_n, bmi_mean_sd,
        edu_low, edu_medium, edu_high, edu_other,
        maxgrip_mean_sd, maxgrip_n, missing_maxgrip,
        walking_mean_sd, walking_n, missing_walking,
        total_missing
    ]


max_len = max(len(v) for v in summary_dict.values())
for k in summary_dict:
    summary_dict[k] += [""] * (max_len - len(summary_dict[k]))

summary_table_maxwalk = pd.DataFrame(summary_dict)
summary_table_maxwalk


,Variable,No disease,Only HT,Only OA,OA and HT,Only Diabetes,HT and Diab,All three diseases,OA and Diab
0,Number of cases,507,377,124,112,77,75,35,24
1,Age; mean (SD),80.2 ± 3.9,80.2 ± 3.9,80.6 ± 3.8,80.4 ± 4.0,79.7 ± 3.3,80.3 ± 3.3,79.6 ± 3.5,78.8 ± 3.2
2,Age; n (%),507 (100.0%),377 (100.0%),124 (100.0%),112 (100.0%),77 (100.0%),75 (100.0%),35 (100.0%),24 (100.0%)
3,"Sex; n, (%)",226.0 (44.6%),212.0 (56.2%),74.0 (59.7%),65.0 (58.0%),38.0 (49.4%),47.0 (62.7%),22.0 (62.9%),10.0 (41.7%)
4,BMI; mean (SD),25.2 ± 4.4,26.1 ± 3.8,26.4 ± 6.5,27.3 ± 3.8,26.9 ± 3.8,27.4 ± 4.1,29.1 ± 4.1,27.6 ± 5.5
5,Educational Level - Low (%),256 (50.5%),203 (53.8%),76 (61.3%),60 (53.6%),48 (62.3%),37 (49.3%),13 (37.1%),14 (58.3%)
6,Educational Level - Medium (%),142 (28.0%),100 (26.5%),17 (13.7%),26 (23.2%),13 (16.9%),19 (25.3%),11 (31.4%),7 (29.2%)
7,Educational Level - High (%),79 (15.6%),53 (14.1%),22 (17.7%),17 (15.2%),8 (10.4%),14 (18.7%),5 (14.3%),0 (0.0%)
8,Educational Level - Other (%),30 (5.9%),21 (5.6%),9 (7.3%),9 (8.0%),8 (10.4%),5 (6.7%),6 (17.1%),3 (12.5%)
9,Grip strength (kg); mean (SD),29.2 ± 9.3,27.4 ± 9.4,24.2 ± 9.1,26.7 ± 10.8,26.8 ± 8.7,25.8 ± 8.1,24.9 ± 8.7,26.3 ± 7.1


##### Create csv from maxchair and maxwalk

In [ ]:
maxchair.to_csv("maxchair.csv")
maxwalk.to_csv("maxwalk.csv")

In [154]:
maxchair.describe()

,age,bmi,female,educational_level,OA_conserv,Hypertension,Diabetes,maxgrip,chair
count,9753.000000,9753.000000,9753.000000,9753.000000,9753.000000,9753.000000,9753.000000,9753.000000,9753.000000
mean,60.091357,26.754493,0.526710,0.900954,0.137496,0.316313,0.082949,37.301958,11.167527
std,6.720434,4.561935,0.499312,0.818064,0.344388,0.465060,0.275819,11.629649,6.903000
min,50.000000,13.061224,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.060000
25%,54.000000,23.875433,0.000000,0.000000,0.000000,0.000000,0.000000,28.000000,7.900000
50%,59.000000,26.234568,1.000000,1.000000,0.000000,0.000000,0.000000,35.000000,10.000000
75%,65.000000,29.043709,1.000000,1.000000,0.000000,1.000000,0.000000,46.000000,12.700000
max,74.000000,157.394159,1.000000,3.000000,1.000000,1.000000,1.000000,84.000000,98.000000


In [155]:
maxwalk.describe()

,age,bmi,female,educational_level,OA_conserv,Hypertension,Diabetes,maxgrip,walking
count,1331.000000,1331.000000,1331.000000,1331.000000,1331.000000,1331.000000,1331.000000,1331.000000,1331.000000
mean,80.207363,26.116982,0.521412,0.754320,0.221638,0.450038,0.158527,27.528926,5.023911
std,3.827951,4.477307,0.499729,0.945374,0.415505,0.497684,0.365372,9.427987,3.886788
min,75.000000,12.802768,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
25%,77.000000,23.437500,0.000000,0.000000,0.000000,0.000000,0.000000,20.500000,2.810000
50%,79.000000,25.711662,1.000000,0.000000,0.000000,0.000000,0.000000,26.000000,3.800000
75%,82.500000,28.344671,1.000000,1.000000,0.000000,1.000000,0.000000,34.000000,5.700000
max,98.000000,80.329225,1.000000,3.000000,1.000000,1.000000,1.000000,80.000000,30.000000


In [156]:
data.describe()

,age,bmi,female,educational_level,OA_conserv,Hypertension,Diabetes,maxgrip,chair,walking,disease_group
count,15821.00000,15821.000000,15821.000000,15821.000000,15821.000000,15821.000000,15821.000000,14414.000000,10513.000000,1425.000000,15821.000000
mean,63.48739,26.769211,0.550913,0.848619,0.180204,0.346754,0.105113,34.973845,11.194354,5.124547,0.632071
std,10.38545,4.703205,0.497417,0.853802,0.384369,0.475952,0.306709,12.022814,7.009656,4.043442,0.762062
min,15.00000,10.816658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
25%,55.00000,23.808690,0.000000,0.000000,0.000000,0.000000,0.000000,26.000000,7.900000,2.825000,0.000000
50%,62.00000,26.218821,1.000000,1.000000,0.000000,0.000000,0.000000,33.000000,10.000000,3.840000,0.000000
75%,71.00000,29.097746,1.000000,1.000000,0.000000,1.000000,0.000000,44.000000,12.720000,5.770000,1.000000
max,100.00000,157.394159,1.000000,3.000000,1.000000,1.000000,1.000000,84.000000,98.000000,30.000000,3.000000


##### Comparison between disease:

In [157]:
maxchair['disease_count'] = maxchair[['OA_conserv', 'Diabetes', 'Hypertension']].sum(axis=1)

summary_chair = maxchair.groupby('disease_count').agg(
    n=('disease_count', 'count'),
    grip_strength_mean=('maxgrip', 'mean'),
    chair_stand_mean=('chair', 'mean')
).round(1)

summary_chair


,n,grip_strength_mean,chair_stand_mean
disease_count,,,
0.0,5617,38.2,10.9
1.0,3125,36.8,11.3
2.0,923,34.3,12.4
3.0,88,31.7,12.7


In [158]:
maxwalk['disease_count'] = maxwalk[['OA_conserv', 'Diabetes', 'Hypertension']].sum(axis=1)

summary_walk = maxwalk.groupby('disease_count').agg(
    n=('disease_count', 'count'),
    grip_strength_mean=('maxgrip', 'mean'),
    walking_mean=('walking', 'mean')
).round(1)

summary_walk 

,n,grip_strength_mean,walking_mean
disease_count,,,
0.0,507,29.2,4.8
1.0,578,26.7,5.1
2.0,211,26.3,5.4
3.0,35,24.9,5.3
